## 二进制乘法

In [1]:
import numpy as np

input1 = 200
input2 = 222

binary_input1 = np.unpackbits(np.array([input1], dtype=np.uint8))
binary_input2 = np.unpackbits(np.array([input2], dtype=np.uint8))

binary_input1=binary_input1[::-1]
binary_input2=binary_input2[::-1]

print(binary_input1)
print(binary_input2)


[0 0 0 1 0 0 1 1]
[0 1 1 1 1 0 1 1]


In [2]:
# 创建一个n*2n的矩阵
n = 8
matrix = np.zeros((n, 2*n), dtype=np.uint8)
print(matrix)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [3]:
#计算乘法
for i in range(n):
    for j in range(n):
        matrix[i,i+j] = binary_input1[j] & binary_input2[i]
print(matrix)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0]]


In [4]:
#reduce_add
def reduce_add(a, b, size):
    sum = [0] * size
    sum[0]=np.bitwise_xor(a[0],b[0])
    carry = np.bitwise_and(a[0],b[0])
    for i in range(1,size):
       tmp_s=np.bitwise_xor(a[i],b[i])
       tmp_c=np.bitwise_and(a[i],b[i])
       sum[i]=np.bitwise_xor(tmp_s,carry)
       carry=np.bitwise_or(np.bitwise_and(tmp_s,carry),tmp_c)
    return sum

result=[0]*2*n
for i in range(n):
    result=reduce_add(result,matrix[i],2*n)

result=result[::-1]
print(result)

[1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0]


## 编译成concrete电路

In [21]:
from concrete import fhe
@fhe.circuit({"binary_input1": "encrypted","binary_input2": "encrypted"})
def circuit2(binary_input1: fhe.tensor[fhe.uint2, 8, ],binary_input2: fhe.tensor[fhe.uint2, 8, ]):
    n = 8
    matrix =fhe.zeros((n, 2*n))
    result=[0]*2*n
    for i in range(n):
        for j in range(n):
            matrix[i,i+j] = np.bitwise_and(binary_input1[j],binary_input2[i])
    for i in range(n):
        result=reduce_add(result,matrix[i],2*n)
    return fhe.array(result)
import time
start=time.time()
circuit2.keygen()
end=time.time()
print("密钥生成时间：",end-start)
enc=circuit2.encrypt(binary_input1,binary_input2)
end=time.time()
print("加密时间：",end-start)
start=time.time()
res_enc=circuit2.run(enc)
end=time.time()
print("计算时间：",end-start)
start=time.time()
result=circuit2.decrypt(res_enc)
end=time.time()
print("解密时间：",end-start)
result=result[::-1]
print(result)


密钥生成时间： 0.6922135353088379
加密时间： 0.6933736801147461
计算时间： 8.589805841445923
解密时间： 0.00011444091796875
[1 0 1 0 1 1 0 1 0 1 1 1 0 0 0 0]


In [22]:
from concrete import fhe
@fhe.circuit({"binary_input1": "encrypted"})
def circuit2(binary_input1: fhe.tensor[fhe.uint2, 8, ]):
    n = 8
    matrix =fhe.zeros((n, 2*n))
    result=[0]*2*n
    for i in range(n):
        for j in range(n):
            matrix[i,i+j] = np.bitwise_and(binary_input1[j],binary_input2[i])
    for i in range(n):
        result=reduce_add(result,matrix[i],2*n)
    return fhe.array(result)
import time
start=time.time()
circuit2.keygen()
end=time.time()
print("密钥生成时间：",end-start)
enc=circuit2.encrypt(binary_input1)
end=time.time()
print("加密时间：",end-start)
start=time.time()
res_enc=circuit2.run(enc)
end=time.time()
print("计算时间：",end-start)
start=time.time()
result=circuit2.decrypt(res_enc)
end=time.time()
print("解密时间：",end-start)
result=result[::-1]
print(result)

密钥生成时间： 0.6841239929199219
加密时间： 0.6849334239959717
计算时间： 5.951915264129639
解密时间： 0.00011920928955078125
[1 0 1 0 1 1 0 1 0 1 1 1 0 0 0 0]


## 将整数转换为二进制数组

In [2]:
def extract_bits_with_shift(num: int) -> list:
  """
  使用移位提取数字的二进制位，并存储到一个数组中。

  Args:
    num: 要提取二进制位的数字。

  Returns:
    二进制位数组。
  """

  bits = []
  for i in range(num.bit_length()):
    bits.append(num >> i & 1)

  return bits


if __name__ == "__main__":
  num = 165
  bits = extract_bits_with_shift(num)
  print(bits)

[1, 0, 1, 0, 0, 1, 0, 1]


## 将二进制数组转换为整数

In [3]:
def binary_to_decimal(binary: list) -> int:
  """
  将二进制数组转换为十进制。

  Args:
    binary: 二进制数组。

  Returns:
    十进制数。
  """

  length = len(binary)
  decimal = 0
  for i in range(length):
    decimal |= binary[i] << i
  return decimal


if __name__ == "__main__":
  binary = [1, 0, 1, 0, 0, 1, 0, 1]
  decimal = binary_to_decimal(binary)
  print(decimal)

165


## 浮点数与定点数转换

In [7]:
#浮点数转定点数
import math

def float_to_fixed(flt: float, maxbits: int, factor: int, clip: bool = True) -> int:
  """
  将浮点数转换为定点数。

  Args:
    flt: 要转换的浮点数。
    maxbits: 定点数的最大位数。
    factor: 定点数的因子。
    clip: 是否裁剪定点数的值。如果 `clip` 为 `true`，则定点数的值将被限制在 `min` 和 `max` 之间。

  Returns:
    定点数的值。
  """

  max = math.pow(2, maxbits - 1) - 1
  min = -max
  fixed = flt * factor

  if clip:
    if fixed > max:
      fixed = max
    elif fixed < min:
      fixed = min

  return int(fixed)

def fixed_to_float(fixed: int, factor: int) -> float:
  """
  将定点数转换为浮点数。

  Args:
    fixed: 要转换的定点数。
    factor: 定点数的因子。

  Returns:
    浮点数的值。
  """

  return fixed / factor

def float_to_fixed_vector(flt: list[float], maxbits: int, factor: int, clip: bool = True) -> list[int]:
  """
  将浮点数数组转换为定点数数组。

  Args:
    flt: 要转换的浮点数数组。
    maxbits: 定点数的最大位数。
    factor: 定点数的因子。
    clip: 是否裁剪定点数的值。如果 `clip` 为 `true`，则定点数的值将被限制在 `min` 和 `max` 之间。

  Returns:
    定点数数组。
  """

  fixed_vector = []
  for f in flt:
    fixed_vector.append(float_to_fixed(f, maxbits, factor, clip))

  return fixed_vector

def fixed_to_float_vector(fixed: list[int], factor: int) -> list[float]:
  """
  将定点数数组转换为浮点数数组。

  Args:
    fixed: 要转换的定点数数组。
    factor: 定点数的因子。

  Returns:
    浮点数数组。
  """

  flt_vector = []
  for f in fixed:
    flt_vector.append(fixed_to_float(f, factor))

  return flt_vector


In [9]:
# 将浮点数转换为定点数
flt = 123.456
maxbits = 8
factor = 10

fixed = float_to_fixed(flt, maxbits, factor)
print(fixed)
# 将定点数转换为浮点数
flt = fixed_to_float(fixed, factor)
print(flt)
# 将浮点数数组转换为定点数数组
flt_vector = [123.456, 789.012]

fixed_vector = float_to_fixed_vector(flt_vector, maxbits, factor)
print(fixed_vector)
# 将定点数数组转换为浮点数数组
flt_vector = fixed_to_float_vector(fixed_vector, factor)
print(flt_vector)

127
12.7
[127, 127]
[12.7, 12.7]
